# MLP Model

In [1]:
## Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:

## Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Feature Data
filename = "features.csv"
output_data = "../../data/output" 
filepath = os.path.join(output_data,filename)

data = pd.read_csv(filepath,index_col=False, header=0)
print(len(data))
data.head()

339


,Solar,Energy,year,Price,REDTI,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2.965,2018.869,1990,7.17,8.8,1,0,0,0,0,0,0,0,0,0,0,0
1,3.196,1640.310,1990,7.48,22.7,0,1,0,0,0,0,0,0,0,0,0,0
2,4.403,1559.289,1990,7.57,29.8,0,0,1,0,0,0,0,0,0,0,0,0
3,4.853,1299.971,1990,7.69,26.8,0,0,0,1,0,0,0,0,0,0,0,0
4,5.357,1143.417,1990,7.96,53.9,0,0,0,0,1,0,0,0,0,0,0,0


In [3]:
def lag_data(lag_count, df):
    features = []
    targets = []
    for i in range(1, lag_count+1):
        df['lag%s' % i] = df.Solar.shift(i)
        df['peek%s' % i] = df.Solar.shift(-1*i)
        features.append('lag%s' % i)
        targets.append('peek%s' % i)
        
    #differencing
    df['diff1'] = df.Solar.diff(1)
    return features, targets, df.iloc[lag_count:-1*lag_count] #dropped rows with nan
    
lag_count = 12
features, targets, data = lag_data(lag_count, data)

print(len(data))

315


In [4]:
print(features, targets)

['lag1', 'lag2', 'lag3', 'lag4', 'lag5', 'lag6', 'lag7', 'lag8', 'lag9', 'lag10', 'lag11', 'lag12'] ['peek1', 'peek2', 'peek3', 'peek4', 'peek5', 'peek6', 'peek7', 'peek8', 'peek9', 'peek10', 'peek11', 'peek12']


## Improve model by adding features

In [5]:
# features.append("Price")
# features.append("Solar")


In [6]:
#targets.append("Solar")
#print(features, targets)

In [7]:
X = data[["year", "Price", "lag1", "peek1", "lag2", "peek2", "lag3", "peek3","lag4","peek4","lag5", "peek5","lag6", "peek6", "lag7", "peek7", "lag8", "peek8", "lag9", "peek9", "lag10","peek10", "lag11","peek11"]] 
y = data["Solar"].values.reshape(-1, 1)
print(X.shape, y.shape)

(315, 24) (315, 1)


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data into Train and Test
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train, X_test = X[1:len(X)-12], X[len(X)-12:]
y_train, y_test = y[1:len(X)-12], y[len(X)-12:]

# Scale the data
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

y_scaler = StandardScaler().fit(y_train)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)
print(len(X_train), len(X_test), len(y_train), len(y_test))

302 12 302 12


In [9]:
from sklearn.neural_network import MLPRegressor
print("Row number of testing data : ", len(X_test))

#------------Model Training-------------------
mlp = MLPRegressor(max_iter=1000, learning_rate_init=0.1, random_state=0, learning_rate='adaptive',
                   activation='relu', solver='adam', tol=0.0, verbose=2 , hidden_layer_sizes = (20,20))


Row number of testing data :  12


In [29]:
import numpy as np
y_train_ravel = np.ravel(y_train_scaled)
train = mlp.fit(X_train_scaled, y_train_ravel)

Iteration 1, loss = 5.24353292
Iteration 2, loss = 2.03086308
Iteration 3, loss = 0.41532253
Iteration 4, loss = 0.40768384
Iteration 5, loss = 0.19137981
Iteration 6, loss = 0.17743926
Iteration 7, loss = 0.17371502
Iteration 8, loss = 0.11268844
Iteration 9, loss = 0.17287228
Iteration 10, loss = 0.10680940
Iteration 11, loss = 0.10896848
Iteration 12, loss = 0.06630323
Iteration 13, loss = 0.08836251
Iteration 14, loss = 0.05431658
Iteration 15, loss = 0.06008242
Iteration 16, loss = 0.03089377
Iteration 17, loss = 0.04792155
Iteration 18, loss = 0.02298896
Iteration 19, loss = 0.03844996
Iteration 20, loss = 0.03312612
Iteration 21, loss = 0.02367918
Training loss did not improve more than tol=0.000000 for two consecutive epochs. Stopping.


In [30]:
y_pred = mlp.predict(X_test_scaled)

In [31]:
mlp.score(X_test_scaled, y_test_scaled)

0.7641087240871138

In [32]:
from sklearn.metrics import mean_squared_error

predictions = mlp.predict(X_test_scaled)
MSE = mean_squared_error(y_test_scaled, predictions)
r2 = mlp.score(X_test_scaled, y_test_scaled)

print(f"MSE: {MSE}, R2: {r2}")

MSE: 0.45891241229818025, R2: 0.7641087240871138


In [34]:
import numpy as np
dataMLP = np.append(y_train_scaled, predictions)
dataMLP

array([-1.13888919, -0.49566738, -0.25613695,  0.01230233,  0.0381138 ,
        0.18110933,  0.15013557, -0.11314141, -0.35473676, -0.80127517,
       -0.90503728, -1.20496655, -1.07745789, -0.41152199, -0.16373189,
        0.11451574,  0.14135967,  0.2895175 ,  0.25699505, -0.01505783,
       -0.26542908, -0.7279706 , -0.83534631, -1.15902213, -1.02789987,
       -0.34337971, -0.08784617,  0.1981449 ,  0.22550506,  0.37830896,
        0.34475405,  0.06444149, -0.19264074, -0.66860422, -0.77907731,
       -1.11307772, -0.97834185, -0.27472121, -0.01247668,  0.28125783,
        0.30965045,  0.46606795,  0.43199681,  0.14394082, -0.12036862,
       -0.60923784, -0.72229208, -1.08623379, -0.94943301, -0.23497155,
        0.03088658,  0.3297834 ,  0.35817601,  0.51717466,  0.48258729,
        0.19040146, -0.07803781, -0.57465048, -0.68976963, -1.07023068,
       -0.93188121, -0.21122499,  0.05773051,  0.35869224,  0.38760109,
        0.54814842,  0.51304482,  0.21827785, -0.05274257, -0.55

In [26]:
dataSolar = np.append(y_train_scaled, y_test_scaled)
dataSolar

array([-1.13888919, -0.49566738, -0.25613695,  0.01230233,  0.0381138 ,
        0.18110933,  0.15013557, -0.11314141, -0.35473676, -0.80127517,
       -0.90503728, -1.20496655, -1.07745789, -0.41152199, -0.16373189,
        0.11451574,  0.14135967,  0.2895175 ,  0.25699505, -0.01505783,
       -0.26542908, -0.7279706 , -0.83534631, -1.15902213, -1.02789987,
       -0.34337971, -0.08784617,  0.1981449 ,  0.22550506,  0.37830896,
        0.34475405,  0.06444149, -0.19264074, -0.66860422, -0.77907731,
       -1.11307772, -0.97834185, -0.27472121, -0.01247668,  0.28125783,
        0.30965045,  0.46606795,  0.43199681,  0.14394082, -0.12036862,
       -0.60923784, -0.72229208, -1.08623379, -0.94943301, -0.23497155,
        0.03088658,  0.3297834 ,  0.35817601,  0.51717466,  0.48258729,
        0.19040146, -0.07803781, -0.57465048, -0.68976963, -1.07023068,
       -0.93188121, -0.21122499,  0.05773051,  0.35869224,  0.38760109,
        0.54814842,  0.51304482,  0.21827785, -0.05274257, -0.55

In [35]:
dfMLP = pd.DataFrame(dataMLP)
dfSolar = pd.DataFrame(dataSolar)

In [37]:
len(dfMLP)

314

In [38]:
len(dfSolar)

314

In [17]:
dataname = "dfMLP.csv"
filesave = os.path.join(output_data,dataname)
dfMLP.to_csv(filesave, index=False)

In [39]:
dataname = "dfSolar.csv"
filesave = os.path.join(output_data,dataname)
dfSolar.to_csv(filesave, index=False)